In [1]:
!pip install pyLDAvis
!pip install nltk
!pip install wordcloud
!pip install TextBlob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 36.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.2/375.2 kB 8.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 757.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 16.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.5 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.1/511.1 kB 11.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('genesis')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from wordcloud import WordCloud
#from IPython.display import display
import base64
import string
import re
import nltk

from collections import Counter
from time import time
# from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package genesis to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Unzipping corpora/genesis.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/codespace/nltk_data...


# Récupération des communications

## Webscrapping

Pour voir notre travail de webscrapping, on pourra se référer au notebook nommé "Essaie webscrapp.ipynb"

## Une première base de donnée : Une centaine d'articles du NYT et du WSJ avec le mot clef environnement sur les derniers jours

In [3]:
# Charger à partir du fichier pickle
data = pd.read_pickle('data.pkl')

# Traitement

## Nettoyage

In [4]:
def preprocess_text(text):
    # Minuscule
    text = text.lower()
    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenization
    words = word_tokenize(text)
    # Suppression des stop-words
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    lemmatized_output = [lemmatizer.lemmatize(w) for w in filtered_words]
    
    return lemmatized_output

# Appliquer la fonction preprocess_text à la colonne 'Article'
data['Preprocessed_Article'] = data['Article'].apply(preprocess_text)

In [5]:
data.head(10)

,Article,Date,Auteur,Nombre de mots,Journal,Titre,ID,Preprocessed_Article
0,\nMetropolitan Desk; SECTMB\nCan an Ambitious ...,31 December 2023,Nick Tabor,529,New York Times,Copyright 2023 The New York Times Company. Al...,Document NYTF000020240104ejcv0000d,"[metropolitan, desk, sectmb, ambitious, public..."
1,\n\nMagazine Desk; SECTMM\nWhen Jim Brown and ...,31 December 2023,Wesley Morris,422,New York Times,"When Jim Brown and Raquel Welch, Two Sexy Star...",Document NYTF000020231231ejcv0006h,"[magazine, desk, sectmm, jim, brown, raquel, w..."
2,\n\nMagazine Desk; SECTMK\nTalking During Movi...,31 December 2023,None,179,New York Times,Talking During Movies: Totally Evil or Part of...,Document NYTF000020231231ejcv00064\n,"[magazine, desk, sectmk, talking, movie, total..."
3,\n\nMagazine Desk; SECTMK\nLet Kids Vote!\n\n4...,31 December 2023,None,454,New York Times,Let Kids Vote!,Document NYTF000020231231ejcv00063\n,"[magazine, desk, sectmk, let, kid, vote, 454, ..."
4,\n\nMagazine Desk; SECTMK\nAre We Doomed to Di...,31 December 2023,Christina Caron,428,New York Times,Are We Doomed to Disagree?,Document NYTF000020231231ejcv0005z\n,"[magazine, desk, sectmk, doomed, disagree, 428..."
5,"\n\nMagazine Desk; SECTMK\nHello, Fourth Grade...",31 December 2023,None,319,New York Times,"Hello, Fourth Graders! A Look Back at our Clas...",Document NYTF000020231231ejcv0005t\n,"[magazine, desk, sectmk, hello, fourth, grader..."
6,\n\nForeign Desk; SECTA\n72 of Our Favorite Fa...,31 December 2023,Times Insider,914,New York Times,72 of Our Favorite Facts From 2023,Document NYTF000020231231ejcv0005r\n,"[foreign, desk, secta, 72, favorite, fact, 202..."
7,\n\nMoney and Business/Financial Desk; SECTBU\...,31 December 2023,Kashmir Hill,811,Stalker Under Your Hood,The Stalker Under Your Hood,Document NYTF000020231231ejcv0005n\n,"[money, businessfinancial, desk, sectbu, stalk..."
8,\n\nBook Review Desk; SECTBR\nPaperback Row\n\...,31 December 2023,Shreya Chattopadhyay,431,New York Times,Paperback Row,Document NYTF000020231231ejcv0005g\n,"[book, review, desk, sectbr, paperback, row, s..."
9,\n\nNICHOLAS KRISTOF\nEditorial Desk; SECTSR\n...,31 December 2023,Nicholas Kristof,976,New York Times,Humans Made Progress In 2023,Document NYTF000020231231ejcv00052\n,"[nicholas, kristof, editorial, desk, sectsr, h..."


## Analyse du sentiment des textes

### Sentiment général

Le score donné varie de -1 à 1 avec -1 comme la négativité maximale et 1 comme la positivité maximale. 0 pour dire que le texte est neutre.

In [7]:
def calculate_sentiment(word_list):
    # Convertir la liste de mots en une chaîne de caractères
    text = ' '.join(word_list)
    # Création d'une instance TextBlob
    analysis = TextBlob(text)
    # Retourner la polarité
    return analysis.sentiment.polarity

# Appliquer la fonction au DataFrame
data['Sentiment'] = data['Preprocessed_Article'].apply(calculate_sentiment)


In [8]:
data["Sentiment"]

0     0.080807
1     0.096043
2     0.129610
3     0.068136
4     0.081156
        ...   
95    0.017752
96    0.062612
97   -0.002605
98    0.129660
99   -0.021999
Name: Sentiment, Length: 100, dtype: float64

### Sentiment environnemental

#### Dictionnaire

L'idéal serait de récupérer un dictionnaire pré existant, spécialisé dans l'évaluation de termes écologique, qui attribue une score à chaque terme. La difficulté à trouver ce type de dictionnaire nous mène dans un premier temps à creuser d'autres pistes de substitution. Nous verrons plus tard si nous réussissons à trouver un dictionnaire préexistant

#### Alternative aux dictionnaires pré-existants

Problème pour l'amélioration du dictionnaire : on ne trouve pas de dictionnaire préexistant avec comme spécialité l'environnement. Deux options : 

 - Améliorer notre dictionnaire fait main:
   - Avantage : On peut contrôler le poid associé à chaque mot, dans la note
   - Inconvéniant : COnstruction peu rigoureuse, on peut avoir oublié des mots
  
 - Utiliser un dictionnaire généraliste :
   - Avantage : Construction plus rigoureuse, moins de chance d'oublier certains termes
   - Inconvénient : Pas de contrôle sur le poid des mots

##### Le dictionnaire fait main

Pour le moment, on ne trouve pas de dictionnaire pré-existant, dont chaque terme peut être associé à une note environnementale. On propose donc de construire nous même un dictionnaire, un en français et l'autre en anglais.

In [ ]:
# Créer un dictionnaire de termes environnementaux positifs
Dico_env_fr = {
    "propre": 1,
    "écologique": 1,
    "durable": 1,
    "vert": 1,
    "économie d'énergie": 1,
    "renouvelable": 1,
    "responsable": 1,
    "conservation": 1,
    "biodiversité": 1,
    "sain": 1,
    "bio": 1,
    "éco-friendly": 1,
    "respectueux de l'environnement": 1,
    "efficace": 1,
    "innovant": 1,
    "éthique": 1,
    "équitable": 1,
    "efficience": 1,
    "responsabilité sociale": 1,
    "sustainable": 1,
    "solidaire": 1,
    "propagation consciente": 1,
    "soutenable": 1,
    "énergie propre": 1,
    "énergie renouvelable": 1,
    "recyclage": 1,
    "efficacité énergétique": 1,
    "économie circulaire": 1,
    "énergie solaire": 1,
    "énergie éolienne": 1,
    "régénération": 1,
    "préservation": 1,
    "restauration": 1,
    "réhabilitation": 1,
    "récupération": 1,
    "restaurateur": 1,
    "régénérateur": 1,
    "revitalisation": 1,
    "positif": 1,
    "bénéfique": 1,
    "valorisation": 1,
    "épanouissement": 1,
    "amélioration continue": 1,
    "prospérité": 1,
    "harmonie": 1,
    "intégrité": 1,
    "consommation responsable": 1,
    "éco-responsable": 1,
    "éco-conscient": 1,
    "durabilité": 1,
    "récupérable": 1,
    "énergie verte": 1,
    "effet de serre": 1,
    "éco-efficace": 1,
    "éco-innovation": 1,
    "bien-être": 1,
    "éco-design": 1,
    "agroécologie": 1,
    "permaculture": 1,
    "éco-citoyen": 1,
    "carbone neutre": 1,
    "zéro déchet": 1,
    "biologique": 1,
    "éco-label": 1,
    "mobilité durable": 1,
    "éco-tourisme": 1,
    "éco-habitat": 1,
    "consommation consciente": 1,

    "pollution": -1,
    "déchet": -1,
    "déforestation": -1,
    "émissions de gaz à effet de serre": -1,
    "contamination": -1,
    "destructeur": -1,
    "irresponsable": -1,
    "gaspillage": -1,
    "nuisible": -1,
    "toxique": -1,
    "détérioration": -1,
    "dégradation": -1,
    "dommageable": -1,
    "préjudiciable": -1,
    "périlleux": -1,
    "inquiétant": -1,
    "catastrophique": -1,
    "catastrophe": -1,
    "dangereux": -1,
    "menace": -1,
    "risque": -1,
    "nocif": -1,
    "néfaste": -1,
    "inadéquat": -1,
    "inapproprié": -1,
    "inopportun": -1,
    "nuire": -1,
    "endommagement": -1,
    "dommages": -1,
    "polluant": -1,
    "polluer": -1,
    "détériorer": -1,
    "perturbation": -1,
    "irrespectueux": -1,
    "malveillant": -1,
    "dégât": -1,
    "agressif": -1,
    "ravageur": -1,
    "gâcher": -1,
    "perturber": -1,
    "endommager": -1,
    "irréparable": -1,
    "toxicité": -1,
    "inacceptable": -1,
    "dommage écologique": -1,
    "abattage illégal": -1,
    "surconsommation": -1,
    "pillage des ressources": -1,
    "dégradation de l'environnement": -1,
    "espace naturel détruit": -1,
    "exploitation excessive": -1,
    "surexploitation": -1,
    "réchauffement climatique": -1,
    "déni environnemental": -1,
}
liste_negation = ["pas", "non","jamais", "aucun", "nul", "rien", "personne", "négatif", "sans", "plus", "moins"]

liste_annulation_negation = ["responsable","à l'origine","la source"]

In [ ]:
Dico_env_en = {
    
    "clean": 1,
    "ecological": 1,
    "sustainable": 1,
    "green": 1,
    "energy-efficient": 1,
    "renewable": 1,
    "responsible": 1,
    "conservation": 1,
    "biodiversity": 1,
    "healthy": 1,
    "organic": 1,
    "eco-friendly": 1,
    "environmentally friendly": 1,
    "efficient": 1,
    "innovative": 1,
    "ethical": 1,
    "fair": 1,
    "efficiency": 1,
    "social responsibility": 1,
    "sustainable": 1,
    "solidarity": 1,
    "conscious spreading": 1,
    "sustainable": 1,
    "clean energy": 1,
    "renewable energy": 1,
    "recycling": 1,
    "energy efficiency": 1,
    "circular economy": 1,
    "solar energy": 1,
    "wind energy": 1,
    "regeneration": 1,
    "preservation": 1,
    "restoration": 1,
    "rehabilitation": 1,
    "recovery": 1,
    "restorer": 1,
    "regenerator": 1,
    "revitalization": 1,
    "positive": 1,
    "beneficial": 1,
    "valorization": 1,
    "fulfillment": 1,
    "continuous improvement": 1,
    "prosperity": 1,
    "harmony": 1,
    "integrity": 1,
    "responsible consumption": 1,
    "eco-responsible": 1,
    "eco-conscious": 1,
    "sustainability": 1,
    "recoverable": 1,
    "green energy": 1,
    "greenhouse effect": 1,
    "eco-efficient": 1,
    "eco-innovation": 1,
    "well-being": 1,
    "eco-design": 1,
    "agroecology": 1,
    "permaculture": 1,
    "eco-citizen": 1,
    "carbon neutral": 1,
    "zero waste": 1,
    "organic": 1,
    "eco-label": 1,
    "sustainable mobility": 1,
    "eco-tourism": 1,
    "eco-habitat": 1,
    "conscious consumption": 1,
    
    "pollution": -1,
    "waste": -1,
    "deforestation": -1,
    "greenhouse gas emissions": -1,
    "contamination": -1,
    "destructive": -1,
    "irresponsible": -1,
    "wasteful": -1,
    "harmful": -1,
    "toxic": -1,
    "deterioration": -1,
    "degradation": -1,
    "damaging": -1,
    "harmful": -1,
    "perilous": -1,
    "worrisome": -1,
    "catastrophic": -1,
    "catastrophe": -1,
    "dangerous": -1,
    "threat": -1,
    "risk": -1,
    "hazardous": -1,
    "harmful": -1,
    "inappropriate": -1,
    "inadequate": -1,
    "inappropriate": -1,
    "harm": -1,
    "damage": -1,
    "pollutant": -1,
    "pollute": -1,
    "deteriorate": -1,
    "disruption": -1,
    "disrespectful": -1,
    "malevolent": -1,
    "damage": -1,
    "aggressive": -1,
    "ravager": -1,
    "spoil": -1,
    "disturb": -1,
    "damage": -1,
    "irreparable": -1,
    "toxicity": -1,
    "unacceptable": -1,
    "ecological damage": -1,
    "illegal logging": -1,
    "overconsumption": -1,
    "resource plundering": -1,
    "environmental degradation": -1,
    "destroyed natural habitat": -1,
    "excessive exploitation": -1,
    "overexploitation": -1,
    "climate change": -1,
    "environmental denial": -1,
}

negation_list = ["not", "no", "never", "none", "nil", "nothing", "nobody", "negative", "without", "more", "less"]

negation_cancellation_list = ["responsible", "originally", "source"]

##### Dictionnaire généraliste

Afin d'explorer la puissance d'un dictionnaire pré exsitant, nous faisant le choix de considérer un dictionnaire pré-existant, même s'il n'est pas spécialisé dans l'environnement

In [ ]:
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('sentiwordnet')

# Termes à examiner
terms = ["good", "bad", "environment", "technology","greenhouse","gases","greenhouse gas"]

for term in terms:
    # Obtenir les synsets associés au terme
    synsets = list(swn.senti_synsets(term))

    if synsets:
        print(f"Scores de sentiment pour le terme '{term}':")
        for synset in synsets:
            print(f"POS: {synset.pos_score()}, NEG: {synset.neg_score()}, OBJ: {synset.obj_score()}")
        print()
    else:
        print(f"Aucun synset trouvé pour le terme '{term}'.\n")

Analyse : On a l'avantage d'avoir trois notes, qui représentent la part de positivité, de négativité et de neutralité du mot => Avancé par rapport à ce qu'on avait proposé. De plus, on considère qu'il y a plusieurs sens à chaque mot, d'où le fait qu'il y ait plusieurs évaluation pour chaque terme
Avantage ; l'algorithme choisi la note du mot en fonction du contexte ?
Problème : les coefficients ne sont pas forcément bons, par exemple, gases n'est jamais négatif, toujours neutre... pourquoi ?

On test maintenant sur une phrase entière :

In [ ]:
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('punkt')

def get_sentiment_scores(term):
    synsets = list(swn.senti_synsets(term))
    if synsets:
        pos_score = sum(s.pos_score() for s in synsets) / len(synsets)
        neg_score = sum(s.neg_score() for s in synsets) / len(synsets)
        obj_score = sum(s.obj_score() for s in synsets) / len(synsets)

        # Normaliser les scores
        total_score = pos_score + neg_score + obj_score
        if total_score != 0:
            pos_score /= total_score
            neg_score /= total_score
            obj_score /= total_score

        return pos_score, neg_score, obj_score
    else:
        return 0, 0, 0

def analyze_sentence(sentence):
    tokens = word_tokenize(sentence)
    scores = []

    for token in tokens:
        pos_score, neg_score, obj_score = get_sentiment_scores(token)
        scores.append((pos_score, neg_score, obj_score))

    # Calculer les scores moyens pour la phrase
    avg_pos_score = sum(score[0] for score in scores) / len(scores)
    avg_neg_score = sum(score[1] for score in scores) / len(scores)
    avg_obj_score = sum(score[2] for score in scores) / len(scores)

    return avg_pos_score, avg_neg_score, avg_obj_score

# Exemple d'utilisation
phrase = "Clean technology promotes sustainable development."
score_phrase = analyze_sentence(phrase)
print("Score de la phrase:", score_phrase)

On remarque un taux de positivité de 16%, contre un taux de négativité de 2%, ainsi qu'un taux de neutralité de 65%. C'est un résultat relativement encourageant, étant donné qu'on a proposé une phrase à l'algorithme qui semblait être positive d'un point de vue environnemental.

On compare ce score au score qu'a la négation de la phrase testée

In [ ]:
phrase = "Clean technology doesn't promotes sustainable development."
score_phrase = analyze_sentence(phrase)
print("Score de la phrase:", score_phrase)

Analyse : Résultats moins encourageant. On observe en effet une baisse du taux de positivité, qui passe de 0.16 à 0.12, en revanche, le taux de négativité n'a pas augmenté, et reste faible, alors même que la phrase semble négative

On test sur une autre phrase négative

In [ ]:
phrase = "Pfizer destroyes environement."
score_phrase = analyze_sentence(phrase)
print("Score de la phrase:", score_phrase)

Score positif très faible, mais le négatif également. Est-ce intéressant de faire un rapport des deux ? On reste incertain quant à la significativité de la construction de notre note

#### Dictionnaire

On suppose que l'on a un dictionnaire `Dico_env` contenant les mots environnementaux, associés avec un score $\in [-1,1]$. Par ex: {'pollution': -1, 'conservation': 1}

#### Colonne environmental_sentiment_score

In [ ]:
def get_environmental_score(token_list, Dico_env):
    score = 0
    token_count = len(token_list)

    for token in token_list:
        if token in Dico_env:
            score += Dico_env[token]

    # Normalize the score to be between -1 and 1
    if token_count > 0:
        normalized_score = score / token_count
        return max(min(normalized_score, 1), -1)
    else:
        return 0

data['environmental_sentiment_score'] = data['Preprocessed_Article'].apply(lambda x: get_environmental_score(x, Dico_env_en))

data.head(10)

Le score environnemental semble être nul pour une partie des articles, on regarde si le score est parfois différent de 0

In [ ]:
somme_valeurs_absolues = data['environmental_sentiment_score'].abs().sum()
print("La somme des valeurs absolue de la variable du score environnement est :", somme_valeurs_absolues)

Ce qui pourrait expliquer le fait qu'aucune / très peu de texte, ai une note différente de 0, est le fait que les éléments du dictionnaire n'ont pas été prétraité (tokenisation etc...)

In [ ]:
# Appliquer la fonction preprocess_text aux clés du dictionnaire
preprocessed_dict_en = {preprocess_text(key): value for key, value in Dico_env_en.items()}
preprocessed_dict_fr = {preprocess_text(key): value for key, value in Dico_env_fr.items()}


# Affichage du dictionnaire après prétraitement des clés
print(preprocessed_dict_en)
print(preprocessed_dict_fr)

Attention, il ne semble pas que la tokenisation ou la lemnisation ait fonctionné...

In [ ]:
def get_environmental_score(token_list, Dico_env):
    score = 0
    token_count = len(token_list)

    for token in token_list:
        if token in Dico_env:
            score += Dico_env[token]

    # Normalize the score to be between -1 and 1
    if token_count > 0:
        normalized_score = score / token_count
        return max(min(normalized_score, 1), -1)
    else:
        return 0

data['environmental_sentiment_score'] = data['Preprocessed_Article'].apply(lambda x: get_environmental_score(x, preprocessed_dict_en))

data.head(10)

In [ ]:
somme_valeurs_absolues = data['environmental_sentiment_score'].abs().sum()
print("La somme des valeurs absolue de la variable du score environnement est :", somme_valeurs_absolues)

Est ce que les éléments de la colonne Processed_Article contiennent une liste de mots qui composent l'article, ou bien c'est un seul élément (une grande chaine de charactère) ? Si c'est une grande chaîne de caractère, ça peut poser problème car pour construire la variable de score envorionnementale, on compare un mot à un texte => Donc le score environnemental reste nul.

In [ ]:
data['Preprocessed_Article'][0]

In [ ]:
data['Preprocessed_Article'][0][0]

Les élements de la variable Processed_Article est donc une longue et unique chaîne de caractère. On créer un code qui prends une chaîne de caractère en entrée, et qui renvoie une liste de mots qui composent cet article

In [ ]:
def mots_dans_article(article):
    # Diviser l'article en mots en utilisant l'espace comme délimiteur
    mots = article.split()

    # Retourner la liste des mots
    return mots

data['Preprocessed_Article_split']=data['Preprocessed_Article'].apply(lambda x: mots_dans_article(x))

data['Preprocessed_Article_split']

In [ ]:
data['environmental_sentiment_score_test'] = data['Preprocessed_Article_split'].apply(lambda x: get_environmental_score(x, preprocessed_dict_en))

data.head(10)

Amélioration du score environnemental => Certains score ne sont pas nul

On essaie de voir maintenant quelles entreprises ont été citées dans le texte :
On peut supposer qu'on a une colonne avec toutes les entreprises qui ont été citées dans le texte, ou dans le titre : entr_citées

In [ ]:
# Importer pandas
import pandas as pd

# Charger le DataFrame des entreprises
df_entreprises = pd.read_csv("Firms.csv")

# Mettre en minuscules les noms des entreprises
df_entreprises['Company'] = df_entreprises['Company'].str.lower()

# Copier la colonne 'Preprocessed_Article' du DataFrame 'data' dans un nouveau DataFrame
df_articles = data[['Preprocessed_Article']].copy()

# Mettre en minuscules le contenu de la colonne 'Preprocessed_Article'
df_articles['Preprocessed_Article'] = df_articles['Preprocessed_Article'].apply(lambda x: x.lower())

# Initialisation de la nouvelle colonne pour stocker les noms des entreprises citées dans chaque article
df_articles['Entreprises_citées'] = ''

# Boucle à travers les articles
for index, article in df_articles.iterrows():
    entreprises_citées = []
    contenu_article = article['Preprocessed_Article']
    
    # Vérifier si le contenu de l'article est une chaîne de caractères
    if isinstance(contenu_article, str):
        # Vérifier la présence de chaque entreprise dans le contenu de l'article
        for index_ent, entreprise in df_entreprises.iterrows():
            nom_entreprise = entreprise['Company']
            if isinstance(nom_entreprise, str) and nom_entreprise in contenu_article:
                entreprises_citées.append(nom_entreprise)
    
    # Stocker les entreprises citées dans la nouvelle colonne
    df_articles.at[index, 'Entreprises_citées'] = ', '.join(entreprises_citées)

# Afficher les articles avec les entreprises citées
print(df_articles[['Preprocessed_Article', 'Entreprises_citées']])


On cherche maintenant à associer à chaque article, une entreprise spécifique (même s'il est cité plusieurs entreprises)

On part du principe qu'un article qui ne cite aucune entreprise n'est pas pertinent à étudier => il n'y aurait aucune communication verte
Si jamais un article ne cite qu'une seule entreprise, alors il ne peut y avoir (ou pas) communication verte que sur cette entreprise (mais pas de communication verte de plusieurs entreprises dans un seul article)
Si jamais il y a plusieurs entreprises citées dans un seul articles, alors on traitera ces données à part, en pensant qu'il peut y avoir plusieurs communications vertes, venant de différentes entreprises, et tout cela dans un seul article.

In [ ]:
# Ajouter une colonne pour le nombre d'entreprises citées dans chaque article
df_articles['Nombre_entreprises_citées'] = df_articles['Entreprises_citées'].apply(lambda x: x.count(',') + 1)

# Compter le nombre d'articles par nombre d'entreprises citées
comptage_entreprises = df_articles['Nombre_entreprises_citées'].value_counts()

# Afficher le comptage des articles par nombre d'entreprises citées
print("Nombre d'articles par nombre d'entreprises citées :")
print(comptage_entreprises)

# Afficher le nombre d'articles qui ne citent aucune entreprise, qui en citent une seule, etc.
print("\nRésumé du nombre d'articles par nombre d'entreprises citées :")
print("Aucune entreprise citée :", comptage_entreprises.get(0, 0))
for i in range(1, max(comptage_entreprises.index) + 1):
    print(f"{i} entreprise(s) citée(s) :", comptage_entreprises.get(i, 0))


In [ ]:
# Suppression des données sans entreprises citées
df_articles = df_articles[df_articles['Entreprises_citées'] != '']

# Identification des articles avec une seule entreprise citée
df_articles['Entreprise_unique'] = df_articles['Entreprises_citées'].apply(lambda x: x.split(',')[0] if ',' not in x else '')

# Affichage des articles avec une seule entreprise citée
articles_avec_entreprise_unique = df_articles[df_articles['Entreprise_unique'] != '']
print(articles_avec_entreprise_unique[['Titre_article', 'Entreprise_unique']])


Si on a une base de données qui contient un certain nomre d'articles deja labellisés avec une note environnementale, on pourrait entrainer un modèle de machine learning plus traditionnel que de l'analyse de sentiment.

en effet, on peut passer par de la vectorization des mots par TF-IDF (ou autre - à voir), puis entrainer un modèle de régression linéaire (ou autre - à voir), et prédire pour les nouveaux articles.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataset['text'])  # 'texts' is the column with your text data
y = dataset['scores']  # 'scores' is the column with your positivity scores

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

new_text = vectorizer.transform(["New text"])
new_score = model.predict(new_text)
print(f'Predicted Sentiment Score: {new_score}')

